# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments            Customer  \
0  Aug 31 2022 12:27PM  245440        12  HH-36679.1           Hush Hush   
1  Aug 31 2022 12:27PM  245440        12  HH-36680.1           Hush Hush   
2  Aug 31 2022 12:27PM  245440        12  HH-36681.1           Hush Hush   
3  Aug 31 2022 12:27PM  245440        12  HH-36682.1           Hush Hush   
4  Aug 31 2022 12:27PM  245440        12  HH-36683.1           Hush Hush   
5  Aug 31 2022 12:21PM  245438        10     8466730  Beach Products Inc   
6  Aug 31 2022 12:17PM  245436        10   PRF-36203             Bio-PRF   
7  Aug 31 2022 12:17PM  245436        10   PRF-36207             Bio-PRF   
8  Aug 31 2022 12:17PM  245436        10   PRF-36210             Bio-PRF   
9  Aug 31 2022 12:17PM  245436        10   PRF-36213             Bio-PRF   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
32  245434       Released          3
33  245435       Released          1
34  245436       Released          4
35  245438       Released          1
36  245440       Released          5

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
245434                NaN        NaN       3.0
245435                NaN        NaN       1.0
245436                NaN        NaN       4.0
245438                NaN        NaN       1.0
245440                NaN        NaN       5.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
244946                0.0       22.0      44.0
245276                0.0        1.0       0.0
245281                0.0        0.0       1.0
245282                0.0        0.0       1.0
245284                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
244946                  0         22        44
245276                  0          1         0
245281                  0          0         1
245282                  0          0         1
245284                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               244946          0         22        44
1               245276          0          1         0
2               245281          0          0         1
3               245282          0          0         1
4               245284          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               244946                  22       44
1               245276                   1         
2               245281                            1
3               245282                            1
4               245284                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                  Customer
0   Aug 31 2022 12:27PM  245440        12                 Hush Hush
5   Aug 31 2022 12:21PM  245438        10        Beach Products Inc
6   Aug 31 2022 12:17PM  245436        10                   Bio-PRF
10  Aug 31 2022 12:14PM  245435        10  ACI Healthcare USA, Inc.
11  Aug 31 2022 12:11PM  245434        10               Emerginnova
14  Aug 31 2022 12:05PM  245433        19       AdvaGen Pharma, Ltd
15  Aug 31 2022 11:40AM  245428        10         ISDIN Corporation
29  Aug 31 2022 11:39AM  245432        22         NBTY Global, Inc.
30  Aug 31 2022 11:39AM  245431        22         NBTY Global, Inc.
31  Aug 31 2022 11:39AM  245430        22         NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                  Customer Completed  \
0  Aug 31 2022 12:27PM  245440        12                 Hush Hush             
1  Aug 31 2022 12:21PM  245438        10        Beach Products Inc             
2  Aug 31 2022 12:17PM  245436        10                   Bio-PRF             
3  Aug 31 2022 12:14PM  245435        10  ACI Healthcare USA, Inc.             
4  Aug 31 2022 12:11PM  245434        10               Emerginnova             
5  Aug 31 2022 12:05PM  245433        19       AdvaGen Pharma, Ltd             
6  Aug 31 2022 11:40AM  245428        10         ISDIN Corporation             
7  Aug 31 2022 11:39AM  245432        22         NBTY Global, Inc.             
8  Aug 31 2022 11:39AM  245431        22         NBTY Global, Inc.             
9  Aug 31 2022 11:39AM  245430        22         NBTY Global, Inc.             

  Executing Released  
0                  5  
1                  1  
2                  4  
3                  1  
4                  3  
5                  1  
6                 14  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                  Customer Released  \
0  Aug 31 2022 12:27PM  245440        12                 Hush Hush        5   
1  Aug 31 2022 12:21PM  245438        10        Beach Products Inc        1   
2  Aug 31 2022 12:17PM  245436        10                   Bio-PRF        4   
3  Aug 31 2022 12:14PM  245435        10  ACI Healthcare USA, Inc.        1   
4  Aug 31 2022 12:11PM  245434        10               Emerginnova        3   
5  Aug 31 2022 12:05PM  245433        19       AdvaGen Pharma, Ltd        1   
6  Aug 31 2022 11:40AM  245428        10         ISDIN Corporation       14   
7  Aug 31 2022 11:39AM  245432        22         NBTY Global, Inc.        1   
8  Aug 31 2022 11:39AM  245431        22         NBTY Global, Inc.        1   
9  Aug 31 2022 11:39AM  245430        22         NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  Aug 31 2022 12:27PM  245440        12                 Hush Hush        5   
1  Aug 31 2022 12:21PM  245438        10        Beach Products Inc        1   
2  Aug 31 2022 12:17PM  245436        10                   Bio-PRF        4   
3  Aug 31 2022 12:14PM  245435        10  ACI Healthcare USA, Inc.        1   
4  Aug 31 2022 12:11PM  245434        10               Emerginnova        3   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Aug 31 2022 12:27PM  245440        12                 Hush Hush       5.0   
1  Aug 31 2022 12:21PM  245438        10        Beach Products Inc       1.0   
2  Aug 31 2022 12:17PM  245436        10                   Bio-PRF       4.0   
3  Aug 31 2022 12:14PM  245435        10  ACI Healthcare USA, Inc.       1.0   
4  Aug 31 2022 12:11PM  245434        10               Emerginnova       3.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Aug 31 2022 12:27PM  245440        12                 Hush Hush       5.0   
1  Aug 31 2022 12:21PM  245438        10        Beach Products Inc       1.0   
2  Aug 31 2022 12:17PM  245436        10                   Bio-PRF       4.0   
3  Aug 31 2022 12:14PM  245435        10  ACI Healthcare USA, Inc.       1.0   
4  Aug 31 2022 12:11PM  245434        10               Emerginnova       3.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2699652      55.0       19.0        4.0
12          245440       5.0        0.0        0.0
15          490365      73.0       23.0        0.0
19          490860      13.0       15.0        0.0
20          245403      18.0        4.0        0.0
21         1962654       6.0        2.0        0.0
22         1227148       5.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2699652      55.0       19.0        4.0
1        12   245440       5.0        0.0        0.0
2        15   490365      73.0       23.0        0.0
3        19   490860      13.0       15.0        0.0
4        20   245403      18.0        4.0        0.0
5        21  1962654       6.0        2.0        0.0
6        22  1227148       5.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      55.0       19.0        4.0
1        12       5.0        0.0        0.0
2        15      73.0       23.0        0.0
3        19      13.0       15.0        0.0
4        20      18.0        4.0        0.0
5        21       6.0        2.0        0.0
6        22       5.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   55.0
1        12  Released    5.0
2        15  Released   73.0
3        19  Released   13.0
4        20  Released   18.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15    19    20   21   22
Status                                          
Completed   4.0  0.0   0.0   0.0   0.0  0.0  0.0
Executing  19.0  0.0  23.0  15.0   4.0  2.0  0.0
Released   55.0  5.0  73.0  13.0  18.0  6.0  5.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15    19    20   21   22
0          Completed   4.0  0.0   0.0   0.0   0.0  0.0  0.0
1          Executing  19.0  0.0  23.0  15.0   4.0  2.0  0.0
2           Released  55.0  5.0  73.0  13.0  18.0  6.0  5.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15    19    20   21   22
0  Completed   4.0  0.0   0.0   0.0   0.0  0.0  0.0
1  Executing  19.0  0.0  23.0  15.0   4.0  2.0  0.0
2   Released  55.0  5.0  73.0  13.0  18.0  6.0  5.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()